# Building `spchcat`

Example script showing how to build the `spchcat` tool, including downloading all data, dependencies, and creating a .deb package installer.

<table class="tfo-notebook-buttons" align="left">
  <td>
      <a target="_blank" href="https://colab.research.google.com/github/petewarden/spchcat/blob/main/notebooks/build.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
      <a target="_blank" href="https://github.com/petewarden/spchcat/blob/main/notebooks/build.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Fetch the Source

We'll grab a copy of the latest code for the tool from GitHub.

In [1]:
!git clone -q https://github.com/petewarden/spchcat
%cd spchcat

/content/spchcat


## Install System Dependencies

We need Sox, Pulse Audio, and some build tools.

In [2]:
!apt-get -qq update
!apt-get -qq install -y sox libsox-dev libpulse-dev make gcc g++ wget curl sudo

Selecting previously unselected package libopencore-amrnb0:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../00-libopencore-amrnb0_0.1.3-2.1_amd64.deb ...
Unpacking libopencore-amrnb0:amd64 (0.1.3-2.1) ...
Selecting previously unselected package libopencore-amrwb0:amd64.
Preparing to unpack .../01-libopencore-amrwb0_0.1.3-2.1_amd64.deb ...
Unpacking libopencore-amrwb0:amd64 (0.1.3-2.1) ...
Selecting previously unselected package libmagic-mgc.
Preparing to unpack .../02-libmagic-mgc_1%3a5.32-2ubuntu0.4_amd64.deb ...
Unpacking libmagic-mgc (1:5.32-2ubuntu0.4) ...
Selecting previously unselected package libmagic1:amd64.
Preparing to unpack .../03-libmagic1_1%3a5.32-2ubuntu0.4_amd64.deb ...
Unpacking libmagic1:amd64 (1:5.32-2ubuntu0.4) ...
Selecting previously unselected package libao-common.
Preparing to unpack .../04-libao-common_1.2.2+20180113-1ubuntu1_all.deb ...
Unpacking libao-common (1.2.2+20180113-1ubuntu1) ...
Selecting previou

## Fetch Coqui Libraries

[Coqui](https://coqui.ai) make binary releases of their [speech to text library](https://github.com/coqui-ai/STT), [TensorFlow Lite](https://www.tensorflow.org/), and [KenLM](https://kheafield.com/code/kenlm/) available for download. This is a lot quicker and easier than building them ourselves from source.

In [3]:
!scripts/download_libs.sh

+ BUILD_DIR=build/
+ LIB_URL=https://github.com/coqui-ai/STT/releases/download/v1.1.0/native_client.tflite.Linux.tar.xz
+ LIB_TMP_DIR=build/lib_tmp/
+ LIB_DIR=build/lib/
+ rm -rf build/lib_tmp/
+ mkdir -p build/lib_tmp/
+ wget -q https://github.com/coqui-ai/STT/releases/download/v1.1.0/native_client.tflite.Linux.tar.xz -O build/lib_tmp/native_client.tflite.Linux.tar.xz
+ unxz build/lib_tmp/native_client.tflite.Linux.tar.xz
+ rm -rf build/lib/
+ mkdir -p build/lib/
+ tar -xf build/lib_tmp/native_client.tflite.Linux.tar --directory build/lib/


## Authenticate to GitHub

The models we need are publicly available from Coqui's GitHub repo as part of their releases. Unfortunately the `gh` tool requires authentication even for publicly-available files, and the auth workflow is quite awkward in a notebook. I've written a helper script that asks for a token from [github.com/settings/tokens/new](https://github.com/settings/tokens/new) and your GitHub username. You'll have to click on the output window to be able to type or cut-and-paste into the script.

In [4]:
!scripts/install_gh.sh

3+1 records in
3+1 records out
1795 bytes (1.8 kB, 1.8 KiB) copied, 0.380464 s, 4.7 kB/s
100 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following NEW packages will be installed:
  gh
0 upgraded, 1 newly installed, 0 to remove and 100 not upgraded.
Need to get 7,368 kB of archives.
After this operation, 30.5 MB of additional disk space will be used.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package gh.
(Reading database ... 155471 files and directories currently installed.)
Preparing to unpack .../archives/gh_2.4.0_amd64.deb ...

## Fetch the Model Files

The tool uses neural network and language models released by Coqui. These are available as downloads from GitHub, but they are quite large so the process of fetching them may take a few minutes.

In [5]:
!scripts/download_models.py

## Build the Tool Code

We build the `spchcat` executable itself, pointing to the libraries we downloaded previously.

In [6]:
!make clean && make spchcat LINK_PATH_STT=-Lbuild/lib EXTRA_CFLAGS_STT=-Ibuild/lib

objdump: '/usr/lib/x86_64-linux-gnu/libmagic.so': No such file
objdump: '/usr/lib/x86_64-linux-gnu/libmagic.so': No such file
rm -f spchcat
objdump: '/usr/lib/x86_64-linux-gnu/libmagic.so': No such file
objdump: '/usr/lib/x86_64-linux-gnu/libmagic.so': No such file
c++   -std=c++11 -o spchcat -Os -Ibuild/lib -I/content/sox-build/include pa_list_devices.cc spchcat.cc  -Wl,--no-as-needed -Wl,-rpath,\$ORIGIN -Lbuild/lib  -lstt -lkenlm -ltflitedelegates -ltensorflowlite -lpulse -l pulse-simple  -L/content/sox-build/lib -lsox


## Create the `.deb` Package

Debian, Ubuntu, and other similar distributions use the `.deb` package format for installing software. This step collects all the libraries, model files, and the executable into a package. In Colab you can download the result by clicking on the folder icon in the left sidebar, and navigating to `spchcat`. The package building process can take a while.

In [ ]:
!scripts/create_deb_package.sh

+ NAME=spchcat
+ VERSION=0.0-2
+ ARCH=amd64
+ MAINTAINER=pete@petewarden.com
+ DESCRIPTION='Speech recognition tool to convert audio to text transcripts.'
+ BUILD_DIR=build/
+ MODELS_DIR=build/models/
+ LIB_URL=https://github.com/coqui-ai/STT/releases/download/v1.1.0/native_client.tflite.Linux.tar.xz
+ LIB_TMP_DIR=build/lib_tmp/
+ LIB_DIR=build/lib/
+ DEB_DIR=build/spchcat_0.0-2_amd64/
+ DEB_MODELS_DIR=build/spchcat_0.0-2_amd64//etc/spchcat/models/
+ DEB_LIB_DIR=build/spchcat_0.0-2_amd64//usr/lib/
+ DEB_BIN_DIR=build/spchcat_0.0-2_amd64//usr/local/bin/
+ DEB_DEBIAN_DIR=build/spchcat_0.0-2_amd64/DEBIAN/
+ DEB_CONTROL_FILE=build/spchcat_0.0-2_amd64/DEBIAN/control
+ rm -rf build/spchcat_0.0-2_amd64/
+ mkdir -p build/spchcat_0.0-2_amd64//etc/spchcat/models/
+ cp -r --verbose build/models/am_ET build/models/bn_IN build/models/br_FR build/models/ca_ES build/models/cnh_MM build/models/cs_CZ build/models/cv_RU build/models/cy_GB build/models/de_DE build/models/dv_MV build/models/el_GR build/mo